# **M3_AI4_PENETRACIÓN Y CRECIMIENTO DE FACEBOOK**
---





- Peredo Mescco, Elizabeth Evelin

In [ ]:
R.version.string
#print(installed.packages())

In [ ]:
library(readr) 
library(tidyr)
library(stringr)
library(dplyr)
library(ggplot2)
install.packages("psych")
library(psych)
install.packages('moments')
library(moments)
install.packages('gridExtra')
library(gridExtra)
install.packages('grid')
library(grid)
install.packages('stargazer')
library(stargazer)
install.packages('kableExtra')
library(kableExtra)
install.packages('scales')
library(scales)
#install.packages('stats')
library(stats)

In [ ]:
getwd()
download.file("https://raw.githubusercontent.com/griu/mbdds_fc20/master/gestion_datos/www/fb_long.csv", "/content/dnld_direct_from_web.csv")

In [ ]:
df <- read.csv("/content/dnld_direct_from_web.csv", sep=";", header=TRUE)

# PALETA DE COLORES Y TEMA

In [ ]:
color1 <- c("#FF8C00", "#008080")
color2 <- c("#fbd1a2", "#70EAAF")

theme <- theme(plot.title = element_text(colour = "black", face = "bold", size = 18, hjust = 0.5),
        axis.title = element_text(colour = "black", face = "bold", size = 12),
        strip.text = element_text(size = 15, face = "bold"),
        axis.text = element_text(colour = "black", size = 12),
        axis.ticks = element_line(colour = "black", linewidth = 0.5),
        panel.grid.major = element_blank(), 
        panel.grid.minor = element_blank(),
        panel.border = element_rect(fill = NA, colour = "black", linewidth = 0.8))

theme1 <- theme(plot.title = element_text(colour = "black", face = "bold", size = 18, hjust = 0.5),
        plot.subtitle = element_text(colour = "black", face = "bold", size = 16, hjust = 0.5),
        axis.title = element_text(colour = "black", size = 14),
        strip.text = element_text(size = 16, face = "bold"),
        axis.text = element_text(colour = "black", size = 12),
        axis.ticks = element_line(colour = "black", linewidth = 0.5),
        panel.background = element_rect(fill = "white", colour = NA),
        panel.grid.major = element_line(colour = "gray80", size = 0.5),
        panel.grid.minor = element_line(colour = "gray90", size = 0.5),
        panel.border = element_rect(fill = NA, colour = "black", linewidth = 0.8),
        #legend.position = "right",
        legend.background = element_blank(),
        legend.title = element_text(size = 14, face = "bold"),
        legend.text = element_text(size = 12),
        legend.key.size = unit(1, "cm"))

# 1. Carga los datos del fichero en un data frame y calcula “Facebook_por_num” = “porcentaje de usuarios de Facebook sobre el total de población”. Presenta el summary de todas las variables del data frame.

- Consejo: procura que Facebook_por_num tenga las mismas unidades de medida que Internet_por_num.

In [ ]:
df_new <- df
head(df_new)

In [ ]:
df_new$Internet_por_num <- ifelse(is.na(df_new$Internet_por), 0, round(as.numeric(str_replace_all(df_new$Internet_por, "\\s|,", ".")), 3))
df_new$Facebook_num <- ifelse(is.na(df_new$Facebook), 0, as.numeric(str_replace_all(df_new$Facebook, "\\s|,", ".")))
df_new$poblacion_num <- as.numeric(gsub(",", ".", gsub("\\.", "", df_new$poblacion)))
df_new$Internet_num <-round(as.numeric(df_new$poblacion_num)*as.numeric(df_new$Internet_por_num)/100)
df_new$Facebook_por_num <- round(as.numeric(df_new$Facebook_num)*100/as.numeric(df_new$poblacion_num),2)

df_new <- df_new[order(df_new$pais_r), ]

head(df_new)

In [ ]:
summary(df_new)

# 2. Haz un análisis univariante de Internet_por_num y Facebook_por_num separado por año. Calcula y comenta el histograma y estadísticos univariantes (centralización, dispersión, posición, concentración y forma).

In [ ]:
df_new <- na.omit(df_new)

df_2010 <- subset(df_new, anyo == 2010)
df_2018 <- subset(df_new, anyo == 2018)

rbind(
  data.frame(Variable="Internet_por_num_2010",x=df_2010$Internet_por_num),
  data.frame(Variable="Internet_por_num_2018",x=df_2018$Internet_por_num),
  data.frame(Variable="Facebook_por_num_2010",x=df_2010$Facebook_por_num),
  data.frame(Variable="Facebook_por_num_2018",x=df_2018$Facebook_por_num)) %>% 
  group_by(Variable)%>%
    
  summarise(
  media=mean(x),
  sd=sd(x),
  CV=sd(x)/mean(x),
  minimo=min(x),
  Q25=quantile(x, 0.25),
  mediana=median(x),
  Q75=quantile(x, 0.75),
  maximo=max(x),
  Skewness = skewness(x), #asimetria
  Kurtosis = kurtosis(x) #apuntamiento
  ) %>% 
    
  pivot_longer (cols =-1, names_to = "Metrica", values_to = "Valor") %>%
    arrange (desc(Variable)) %>%
    pivot_wider (names_from=Variable, values_from= Valor)

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 5)

Facebook <- ggplot(df_new, aes(Facebook_por_num)) + 
  geom_histogram(
    aes(colour = factor(anyo), 
    fill = factor(anyo)), 
    bins = nclass.Sturges(df_new$Facebook_por_num)) +
  facet_grid(~ anyo) + 
  scale_colour_manual(values = color1) +
  scale_fill_manual(values = color2) +
  xlab("% usuarios de Facebook") +
  ylab("Frecuencia") +
  ggtitle("Facebook_por_num") +
  theme1 + guides(colour = "none", fill = "none")

Internet <- ggplot(df_new, aes(Internet_por_num)) + 
  geom_histogram(
    aes(colour = factor(anyo), 
    fill = factor(anyo)), 
    bins = nclass.Sturges(df_new$Internet_por_num)) +
  facet_grid(~ anyo) + 
  scale_colour_manual(values = color1) +
  scale_fill_manual(values = color2) +
  xlab("% usuarios de Internet") +
  ylab("Frecuencia") +
  ggtitle("Internet_por_num") +
  theme1 + guides(colour = "none", fill = "none")

grid.arrange(Facebook, Internet, nrow = 1, ncol = 2)

# 3. Haz un gráfico para cada variable (Facebook_por_num y Internet_por_num) donde se pueda visualizar la variación entre 2010 y 2018 del valor medio (o el estadístico que prefieras). Explica qué observas en el gráfico.

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 5)

Facebook <- ggplot(df_new, aes(Facebook_por_num)) + 
  geom_boxplot(aes(colour = factor(anyo), fill = factor(anyo))) + coord_flip() +
  facet_grid(~ anyo) + 
  scale_colour_manual(values = color1) +
  scale_fill_manual(values = color2) +
  ylab("") + xlab("% usuarios de Facebook") +
  ggtitle("Facebook_por_num") +
  theme1 + guides(colour = "none", fill = "none")

Internet <- ggplot(df_new, aes(Internet_por_num)) + 
  geom_boxplot(aes(colour = factor(anyo), fill = factor(anyo))) + coord_flip() +
  facet_grid(~ anyo) + 
  scale_colour_manual(values = color1) +
  scale_fill_manual(values = color2) +
  ylab("") + xlab("% usuarios de Internet") +
  ggtitle("Internet_por_num") +
  theme1 + guides(colour = "none", fill = "none") 

grid.arrange(Facebook, Internet, nrow = 1, ncol = 2)

# 4. Haz un análisis bivariante entre Facebook_por_num sobre Internet_por_num:
- Gráfico de dispersión separando el año de observación en colores distintos.
- Introduce, en el gráfico de dispersión, una recta de regresión (una por año).
- Comenta si crees que la relación observada es lineal.

In [ ]:
df_new$Facebook_por_Internet <- round(ifelse(df_new$Internet_num > 0, 
                                             as.numeric(df_new$Facebook_num) * 100 / as.numeric(df_new$Internet_num), 
                                             0), 2)


In [ ]:
options(repr.plot.width = 10, repr.plot.height = 7)

df_plot <- df_new[, c("Facebook_por_num", "Internet_por_num", "anyo")]

ggplot(df_plot, aes(
  y = Facebook_por_num, 
  x = Internet_por_num, 
  color = factor(anyo))) +
  geom_point() +
  geom_smooth(method = "lm", se = FALSE) +
  scale_color_manual(values = c("#FF8C00", "#008080"))+
  theme1 +
  labs(y = "% usuarios de Facebook", x = "% usuarios de Internet") +
  ggtitle("Relación entre Facebook y el uso de Internet") 


**COMENTARIO:**
El grafico precedente muestra que las variables Facebook_por_num y Internet_por_num tienen una correlación positiva entre ambas en los dos años 2010 y 2018. Lo que muestra una relación significativa y positiva entre Facebook_por_num e Internet_por_num en ambos años.

# 5. Propón alguna transformación sobre las variables del análisis y argumenta cuál es la mejor.

Segun el punto 3, se puede evidenciar que las variables en los diferentes años 2010 y 2018 hay variaciones en la distribución de datos. Para ello, se utilizaron las siguiente transformaciones según la asimetría de cada variable y de cada año. Al aplicar estas transformaciones, se espera que las variables tengan una distribución más cercana a la normalidad, lo que puede mejorar la precisión del modelo de regresión lineal:

- No realizamos ninguna transformación en la variable `Internet_por_num` del año `2010`, ya que su distribución parece ser cercana a una normal.
- En la variable `Internet_por_num` del año `2018`, observamos una asimetría negativa en su distribución, por lo que elevamos al cuadrado la variable (x^2).
- Para la variable `Facebook_por_num` del año `2010`, observamos una asimetría positiva, por lo que aplicamos la transformación de la raíz cuadrada (sqrt(x)).
- En la variable `Facebook_por_num` del año `2018`, observamos una asimetría negativa en su distribución, por lo que elevamos al cuadrado la variable (x^2).

Adicionalmente se uso scale para tener las variables dentro del rango de 0 a 100

In [ ]:
# Transformación de Internet_por_num y Facebook_por_num según el año
df_trans1 <- na.omit(df_new)

df_trans1$Internet_por_num_t <- ifelse(df_trans1$anyo == 2010, df_trans1$Internet_por_num,
                                    df_trans1$Internet_por_num^2)

df_trans1$Facebook_por_num_t <- ifelse(df_trans1$anyo == 2010, rescale(sqrt(df_trans1$Facebook_por_num),to=c(0,100)),
                                    rescale(df_trans1$Facebook_por_num^2,to=c(0,100)))

df_trans1$Internet_por_num_t2 <-ifelse(df_trans1$anyo == 2010, df_trans1$Internet_por_num_t, rescale(df_trans1$Internet_por_num_t,to=c(0,100)))
df_trans1$Facebook_por_num_t2 <-rescale(df_trans1$Facebook_por_num_t,to=c(0,100))

summary(df_trans1[c('Facebook_por_num_t','Internet_por_num_t','Facebook_por_num_t2','Internet_por_num_t2')])

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)

h_Facebook <- ggplot(df_trans1, aes(Facebook_por_num_t2)) + 
  geom_histogram(
    aes(colour = factor(anyo), 
    fill = factor(anyo)), 
    bins = nclass.Sturges(df_trans1$Facebook_por_num_t2)) +
  facet_grid(~ anyo) + 
  scale_colour_manual(values = color1) +
  scale_fill_manual(values = color2) +
  xlab("% usuarios de Facebook") +
  ylab("Frecuencia") +
  ggtitle("Facebook_por_num_t2") +
  theme1 + guides(colour = "none", fill = "none")

h_Internet <- ggplot(df_trans1, aes(Internet_por_num_t2)) + 
  geom_histogram(
    aes(colour = factor(anyo), 
    fill = factor(anyo)), 
    bins = nclass.Sturges(df_trans1$Internet_por_num_t2)) +
  facet_grid(~ anyo) + 
  scale_colour_manual(values = color1) +
  scale_fill_manual(values = color2) +
  xlab("% usuarios de Internet") +
  ylab("Frecuencia") +
  ggtitle("Internet_por_num_t2") +
  theme1 + guides(colour = "none", fill = "none")


bx_Facebook <- ggplot(df_trans1, aes(Facebook_por_num_t2)) + 
  geom_boxplot(aes(colour = factor(anyo), fill = factor(anyo))) + coord_flip() +
  facet_grid(~ anyo) + 
  scale_colour_manual(values = color1) +
  scale_fill_manual(values = color2) +
  ylab("") + xlab("% usuarios de Facebook") +
  ggtitle("Facebook_por_num_t2") +
  theme1 + guides(colour = "none", fill = "none")

bx_Internet <- ggplot(df_trans1, aes(Internet_por_num_t2)) + 
  geom_boxplot(aes(colour = factor(anyo), fill = factor(anyo))) + coord_flip() +
  facet_grid(~ anyo) + 
  scale_colour_manual(values = color1) +
  scale_fill_manual(values = color2) +
  ylab("") + xlab("% usuarios de Internet") +
  ggtitle("Internet_por_num_t2") +
  theme1 + guides(colour = "none", fill = "none") 

grid.arrange(h_Facebook, h_Internet, bx_Facebook, bx_Internet, nrow = 2, ncol = 2)

In [ ]:
df_trans1 <- na.omit(df_trans1)

df_2010 <- subset(df_trans1, anyo == 2010)
df_2018 <- subset(df_trans1, anyo == 2018)

df_metric_t <- rbind(
                  data.frame(Variable="Int_por_2010",x=df_2010$Internet_por_num),
                  data.frame(Variable="Int_por_2018",x=df_2018$Internet_por_num),
                  data.frame(Variable="Fac_por_2010",x=df_2010$Facebook_por_num),
                  data.frame(Variable="Fac_por_2018",x=df_2018$Facebook_por_num),
                  data.frame(Variable="Int_por_t_2010",x=df_2010$Internet_por_num_t2),
                  data.frame(Variable="Int_por_t_2018",x=df_2018$Internet_por_num_t2),
                  data.frame(Variable="Fac_por_t_2010",x=df_2010$Facebook_por_num_t2),
                  data.frame(Variable="Fac_por_t_2018",x=df_2018$Facebook_por_num_t2)) %>% 
                  group_by(Variable)%>%
                    
                  summarise(
                  media=mean(x),
                  sd=sd(x),
                  CV=sd(x)/mean(x),
                  minimo=min(x),
                  Q25=quantile(x, 0.25),
                  mediana=median(x),
                  Q75=quantile(x, 0.75),
                  maximo=max(x),
                  Skewness = skewness(x), #asimetria
                  Kurtosis = kurtosis(x) #apuntamiento
                  ) %>% 
                    
                  pivot_longer (cols =-1, names_to = "Metrica", values_to = "Valor") %>%
                    arrange (desc(Variable)) %>%
                    pivot_wider (names_from=Variable, values_from= Valor)

df_metric_t <- df_metric_t[c('Metrica','Int_por_2010','Int_por_t_2010','Int_por_2018','Int_por_t_2018','Fac_por_2010','Fac_por_2018','Fac_por_t_2010','Fac_por_t_2018')]
df_metric_t

# 6. Compara, mediante stargazer, los coeficientes de la recta de regresión de Facebook_por_num sobre Internet_por_num en 2010 y 2018. Interpreta el significado de los coeficientes de la regresión en cada año.

In [ ]:
model_2010 <- lm(Facebook_por_num ~ Internet_por_num, data = subset(df_new, anyo == 2010))
model_2018 <- lm(Facebook_por_num ~ Internet_por_num, data = subset(df_new, anyo == 2018))

In [ ]:
summary(model_2010)

**La ecuación del modelo para el año 2010 es:**

`Facebook_por_num = -0.37314 + 0.41329 * Internet_por_num`

- Los resultados muestran que la relación entre las dos variables es significativa (p < 0.001), con un coeficiente de regresión de 0.41329, lo que indica que por cada unidad adicional de Internet_por_num se espera un aumento de 0.41329 en Facebook_por_num.
- El coeficiente de determinación R-cuadrado es 0.4673, lo que significa que el modelo explica el 46.73% de la variabilidad en la variable Facebook_por_num, lo que indica que el modelo se ajusta adecuadamente a los datos.
- El error estándar residual es 12.26, lo que significa que en promedio, las predicciones del modelo están dentro de 12.26 de las observaciones reales.
- El valor F de la prueba de significancia global es 66.67 con un p-valor muy bajo, lo que indica que el modelo es significativo.

In [ ]:
summary(model_2018)

**La ecuación del modelo para el año 2018 es:**

`Facebook_por_num = -3.91856 + 0.72047 * Internet_por_num`

- La variable explicativa Internet_por_num tiene un coeficiente de regresión estimado de 0.72047 y un valor p muy bajo, lo que sugiere una fuerte asociación entre Internet_por_num y Facebook_por_num.
-El modelo tiene un R2 ajustado de 0.4823, lo que significa que el 48.23% de la variación en Facebook_por_num se puede explicar por la variable Internet_por_num en el modelo. 
-El F estadístico es 72.74 y tiene un valor p muy bajo, lo que sugiere que el modelo es significativo en general.


### ANALISIS DE AMBOS MODELOS
Los dos cuadros presentan un modelo de regresión lineal simple en el que se busca explicar la relación entre la cantidad de usuarios de Facebook y la cantidad de usuarios de Internet en los años 2010 y 2018.

En ambos casos, se encuentra que la variable explicativa Internet_por_num es significativa para explicar la variable respuesta Facebook_por_num, lo que se demuestra por la p-value muy baja. La pendiente de la recta de regresión es positiva en ambos casos, lo que indica que a medida que aumenta el número de usuarios de Internet, también lo hace el número de usuarios de Facebook.

El modelo del año 2010 tiene un R-cuadrado ajustado de 0.46, lo que indica que el modelo explica aproximadamente el 46% de la variabilidad en la variable respuesta. Por otro lado, el modelo del año 2018 tiene un R-cuadrado ajustado de 0.48, lo que indica que el modelo explica aproximadamente el 48% de la variabilidad en la variable respuesta.

Cabe destacar que el modelo del año 2018 presenta una mayor magnitud de los residuos respecto a la variable respuesta, lo que podría indicar una mayor dispersión de los datos en este año en comparación con el año 2010.

# 7. Compara, mediante stargazer, los coeficientes de la recta de regresión de Facebook_por_num sobre Internet_por_num en 2010 y 2018. Interpreta el significado de los coeficientes de la regresión en cada año.

In [ ]:
stargazer(model_2010, model_2018, type = "text", title = "Coeficientes del modelo de regresión de Facebook_por_num - Internet_por_num")

La tabla precedente muestra los coeficientes de las rectas de regresión para la variable dependiente Facebook_por_num en función de la variable independiente Internet_por_num, en dos años distintos (2010 y 2018).

Para el año 2010, el modelo indica que por cada aumento de una unidad en Internet_por_num, Facebook_por_num aumenta en 0.413 unidades. Este coeficiente es significativo con un valor p < 0.01, lo que indica que es altamente improbable que se haya obtenido este resultado por azar. El valor del intercepto (constante) no es significativo, lo que indica que no hay una relación significativa entre Facebook_por_num e Internet_por_num cuando esta última variable tiene un valor de cero.

Para el año 2018, el modelo indica que por cada aumento de una unidad en Internet_por_num, Facebook_por_num aumenta en 0.720 unidades. Este coeficiente también es significativo con un valor p < 0.01. El valor del intercepto es nuevamente no significativo.

En cuanto al coeficiente de determinación (R2), se observa que en ambos años es relativamente similar (0.467 y 0.489 respectivamente), lo que indica que el modelo explica alrededor del 46% al 49% de la variabilidad de Facebook_por_num. El valor del estadístico F es significativo en ambos casos, lo que indica que la regresión es significativa y que hay una relación significativa entre las dos variables.

### ANALISIS CON DATOS TRANSFORMADOS

In [ ]:
model_2010_t <- lm(Facebook_por_num_t2 ~ Internet_por_num_t2, data = subset(df_trans1, anyo == 2010))
model_2018_t <- lm(Facebook_por_num_t2 ~ Internet_por_num_t2, data = subset(df_trans1, anyo == 2018))

In [ ]:
stargazer(model_2010_t, model_2018_t, type = "text", title = "Coeficientes del modelo de regresión de Facebook_por_num - Internet_por_num")

En este análisis se aplicó una transformación cuadrática a las variables predictoras antes de ajustar los modelos de regresión lineal. Se utilizaron los mismos datos que en el análisis anterior, pero con las variables transformadas.

Los coeficientes de regresión muestran una relación positiva y significativa entre Facebook_por_num_t2 e Internet_por_num_t2 en ambos años (2010 y 2018), lo que indica que a medida que aumenta Internet_por_num_t2, también lo hace Facebook_por_num_t2.

Sin embargo, el coeficiente de determinación (R-cuadrado) para el año 2010 (0.509) es mayor que para el año 2018 (0.443), lo que indica que el modelo de regresión es más explicativo para el año 2010 que para el año 2018. Además, el modelo de regresión para el año 2010 tiene un F-statistic mayor que el del año 2018, lo que indica que el modelo de regresión es más significativo en el año 2010 que en el año 2018.